# Analyse 

In [20]:
import cPickle as pkl
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess

In [9]:
filepath_0000 = "../results/polyphasic cell cycle from 0 to 300, updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20160119_1705_0600s.p"
filepath_1200 = "../results/polyphasic cell cycle from 1200 to 1500, updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20160120_1149_0600s.p"

Function to replace _numpypy.multiarray by numpy.core.multiarray. Source: http://stackoverflow.com/questions/39086/search-and-replace-a-line-in-a-file-in-python

In [10]:
from tempfile import mkstemp
from shutil import move
from os import remove, close

def replace(file_path, pattern, subst):
    #Create temp file
    fh, abs_path = mkstemp()
    with open(abs_path,'w') as new_file:
        with open(file_path) as old_file:
            for line in old_file:
                new_file.write(line.replace(pattern, subst))
    close(fh)
    #Remove original file
    remove(file_path)
    #Move new file
    move(abs_path, file_path)

In [11]:
replace(filepath_0000, "_numpypy.multiarray", "numpy.core.multiarray")
replace(filepath_1200, "_numpypy.multiarray", "numpy.core.multiarray")

In [12]:
results_0000 = pkl.load(open(filepath_0000))
results_1200 = pkl.load(open(filepath_1200))

In [29]:
results_0000_cut = {key: results_0000['proteome'][key] for key in results_0000['proteome'] if results_0000['proteome'][key] > 1e-1}
results_1200_cut = {key: results_1200['proteome'][key] for key in results_1200['proteome'] if results_1200['proteome'][key] > 1e-1}

In [30]:
def scatter_dicts(dict1, dict2, xaxislabel=None, yaxislabel=None, fontsize=None):
    plotkeys = dict1.viewkeys() & dict2.viewkeys() # set intersection
    print len(plotkeys)
    xs = [dict1[key] for key in plotkeys]
    ys = [dict2[key] for key in plotkeys] 

    fig =  plt.figure()                

    ax = fig.add_subplot(111)
    ax.grid(True, linestyle = '-', color = '0.75')
    ax.set_xscale('log')
    ax.set_yscale('log')
    if xaxislabel:
        ax.set_xlabel(xaxislabel)
        if fontsize:
            ax.set_xlabel(xaxislabel, fontsize=fontsize)
            plt.setp(ax.get_xticklabels(), fontsize=fontsize)
    if yaxislabel:
        ax.set_ylabel(yaxislabel)
        if fontsize:
            ax.set_ylabel(yaxislabel, fontsize=fontsize)
            plt.setp(ax.get_yticklabels(), fontsize=fontsize)

    scat = plt.scatter(xs, ys)
    scat.set_alpha(0.2)

    points = zip(xs, ys)
    sorted_points = sorted(points)
    new_xs = [point[0] for point in sorted_points]
    new_ys = [point[1] for point in sorted_points]
    ylowess = lowess(new_ys, new_xs)[:,1]
    plt.plot(new_xs, ylowess, 'orange', linewidth=4)

    plt.show()

In [31]:
scatter_dicts(results_0000_cut, results_1200_cut, xaxislabel="proteome (300 s)", yaxislabel="proteome (1500 s)", fontsize=None)

527
